In [5]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define data paths
base_dir = "/Users/swastika/Fruit_Vegetable_Recognition/archive-3"
train_dir = os.path.join(base_dir, "train")
validation_dir = os.path.join(base_dir, "validation")
test_dir = os.path.join(base_dir, "test")

# Set parameters
image_size = (150, 150)  # Resize images to 150x150
batch_size = 32
num_classes = 36  # Number of classes (carrot, apple, radish, etc.)

# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Only rescaling for validation and test sets
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(num_classes, activation="softmax"),
])

# Compile the model
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

# Display model summary
model.summary()

# Train the model
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=20,
    callbacks=[early_stopping],
)

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Save the model
model.save("fruit_vegetable_classifier.h5")


Found 3115 images belonging to 36 classes.
Found 351 images belonging to 36 classes.
Found 359 images belonging to 36 classes.


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_11 (Conv2D)              │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 36)             │         4,644 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,832,996 (18.44 MB)

...

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0938 - loss: 3.2501 - val_accuracy: 0.0323 - val_loss: 3.2152
Epoch 3/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 95s 975ms/step - accuracy: 0.0987 - loss: 3.2430 - val_accuracy: 0.2250 - val_loss: 2.5446
Epoch 4/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0312 - loss: 3.2107 - val_accuracy: 0.3226 - val_loss: 2.5350
Epoch 5/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 72s 737ms/step - accuracy: 0.1432 - loss: 2.9556 - val_accuracy: 0.2844 - val_loss: 2.3627
Epoch 6/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2188 - loss: 2.8315 - val_accuracy: 0.2581 - val_loss: 2.3798
Epoch 7/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 67s 689ms/step - accuracy: 0.1571 - loss: 2.8466 - val_accuracy: 0.3344 - val_loss: 2.1728
Epoch 8/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0938 - loss: 3.0645 - val_accuracy: 0.3548 - val_loss: 1.9917
Epoch 9/20


2024-11-19 23:11:03.233117: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


97/97 ━━━━━━━━━━━━━━━━━━━━ 64s 653ms/step - accuracy: 0.1905 - loss: 2.6847 - val_accuracy: 0.3750 - val_loss: 2.0065
Epoch 10/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2812 - loss: 2.7100 - val_accuracy: 0.1935 - val_loss: 2.0487
Epoch 11/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 67s 691ms/step - accuracy: 0.2022 - loss: 2.6114 - val_accuracy: 0.3688 - val_loss: 2.0398
Epoch 12/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2500 - loss: 2.3888 - val_accuracy: 0.4194 - val_loss: 1.8027
Epoch 13/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 72s 745ms/step - accuracy: 0.2247 - loss: 2.5681 - val_accuracy: 0.3656 - val_loss: 2.0503
Epoch 14/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2500 - loss: 2.5750 - val_accuracy: 0.4516 - val_loss: 1.8770
Epoch 15/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 70s 722ms/step - accuracy: 0.2139 - loss: 2.5296 - val_accuracy: 0.3969 - val_loss: 1.7923
Epoch 16/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1562 - loss: 2.7316 - val_accuracy: 0.3548 - v

Test Accuracy: 47.63%


In [8]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img,img_to_array
print(tf.__version__)

2.18.0


In [27]:
# Predict the label of the test_images
pred = model.predict(test_datagen)
pred = np.argmax(pred,axis=1)
# Map the label
labels = (train_datagen.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred1 = [labels[k] for k in pred]
pred1

ValueError: Unrecognized data type: x=<keras.src.legacy.preprocessing.image.ImageDataGenerator object at 0x30db5cb50> (of type <class 'keras.src.legacy.preprocessing.image.ImageDataGenerator'>)

In [36]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def predict_single_image(image_path, model, class_indices):
    """
    Predicts the class of a single image.

    Parameters:
    - image_path (str): Path to the image to be predicted.
    - model (keras.Model): Trained Keras model.
    - class_indices (dict): Mapping of class names to their indices.

    Returns:
    - str: Predicted class name.
    """
    # Load and preprocess the image
    image = load_img(image_path, target_size=(150, 150))  # Resize to match model input size
    image_array = img_to_array(image)  # Convert to NumPy array
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
    image_array = image_array / 255.0  # Normalize to match training preprocessing

    # Predict the class
    predictions = model.predict(image_array)
    predicted_class_index = np.argmax(predictions)
    class_labels = {v: k for k, v in class_indices.items()}  # Reverse the class_indices
    predicted_class_name = class_labels[predicted_class_index]

    return predicted_class_name

# Example usage after training the model
if __name__ == "__main__":
    # Path to a single image
    single_image_path = "/Users/swastika/Fruit_Vegetable_Recognition/istockphoto-1388403435-612x612.jpg"

    # Get class indices from the training generator
    class_indices = train_generator.class_indices

    # Predict the class
    predicted_class = predict_single_image(single_image_path, model, class_indices)
    print(f"The predicted class is: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
The predicted class is: carrot
